# **LISTA 5**

**Edvonaldo Horácio e José Carlos Pinheiro**

1. O objetivo dessa questão  ́e desenvolver um buscador de documentos.


In [ ]:
folder_path = '/content/tech'
#folder_path = '/content/drive/My Drive/tech'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
import nltk
from nltk.tokenize import RegexpTokenizer
import glob
import pandas as pd
import matplotlib.pyplot as plt

# **Pré processamento**

In [ ]:
## Pré processamento

texts = []


for filename in glob.glob(os.path.join(folder_path, '*.txt')):
	with open(filename, 'r') as f_in:
		texts.append([filename.split('/')[-1].split('.')[0], f_in.read()])

df = pd.DataFrame.from_records(texts)
df.columns = ["id", "content"]
print(df.tail())

ValueError: ignored

In [ ]:
## Removendo palavras com números
df = df.replace(to_replace=r'\w*\d\w*', value='', regex=True)

In [ ]:
## Removendo pontuação

import re
import string
def remove_punctuation(text):
  return  re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
'''
df['content'] = df['content'].apply(remove_punctuation) # Removendo pontuação
'''


"\ndf['content'] = df['content'].apply(remove_punctuation) # Removendo pontuação\n"

In [ ]:
# Converter todas as palavras em minúsculas
for i in range(df['content'].shape[0]):
  df['content'][i] = df['content'][i].lower()

In [ ]:
df['content']

0      satellite mapping aids darfur relief\n\naid wo...
1      bond game fails to shake or stir\n\nfor gaming...
2      gamer buys $, virtual land\n\na -year-old game...
3      latest opera browser gets vocal\n\nnet browser...
4      peer-to-peer nets 'here to stay'\n\npeer-to-pe...
                             ...                        
396    eu software patent law delayed\n\ncontroversia...
397    musicians 'upbeat' about the net\n\nmusicians ...
398    'blog' picked as word of the year\n\nthe term ...
399    napster offers rented music to go\n\nmusic dow...
400    broadband in the uk gathers pace\n\none person...
Name: content, Length: 401, dtype: object

In [ ]:
for i in range(df['content'].shape[0]):
  df['content'][i] = df['content'][i].replace("\n", " ")

df['content']

0      apple unveils low-cost 'mac mini'  apple has u...
1      commodore finds new lease of life  the once-fa...
2      sony psp tipped as a 'must-have'  sony's plays...
3      ink helps drive democracy in asia  the kyrgyz ...
4      ask jeeves joins web log market  ask jeeves ha...
                             ...                        
396    multi-purpose tv aids india  two-thirds of the...
397    us state acts to stop 'spammers'  us state tex...
398    new delay hits eu software laws  a fresh delay...
399    hacker threat to apple's itunes  users of appl...
400    apple mac mini gets warm welcome  the mac mini...
Name: content, Length: 401, dtype: object

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize 
import nltk
nltk.download('punkt')


data = []
for text in range(df['content'].shape[0]):
  data_aux = []
  for i in sent_tokenize(df['content'][text]): 
      temp = [] 

      # tokenize sentença para palavra 
      for j in word_tokenize(remove_punctuation(i)): ## Retirando a pontuação
          
          temp.append(j.lower()) 

      data_aux.append(temp) 
  data.append((text, data_aux))   
  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
print(len(data))
data[0][1][0][0]

401


'apple'

# **Treinamento**

In [ ]:
import gensim 
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)

common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [ ]:
import gensim.downloader as api
word_vectors50 = api.load("glove-wiki-gigaword-50")
word_vectors100 = api.load("glove-wiki-gigaword-100")
word_vectors200 = api.load("glove-wiki-gigaword-200")

models = [(word_vectors50, 'Com 50'), (word_vectors100, 'Com 100'), (word_vectors200, 'Com 200')]

#models = [(word_vectors50, 'Com 50')]


[==================================================] 100.0% 128.1/128.1MB downloaded
[==================================================] 100.0% 252.1/252.1MB downloaded


# Primeira questão

In [ ]:
words_to_compare = ['walmart', 'pineapple', 'platypus', 'python', 'javascript']
for model in models:
  print('***************** Resultados ' + model[1] + ' *****************')
  for word in words_to_compare:
    result = []
    for docum in range(0, len(data)): ##Por documento
      for vects in range(0, len(data[docum][1])): ## Vetor de vetores de palavras
        for arr in range(0, len(data[docum][1][vects])): ## Vetor de palavra
          try:
            word_acc = data[docum][1][vects][arr] ## palavra
            similarity = model[0].similarity(word, word_acc)
            result.append((data[docum][1][vects][arr], docum, similarity))
          except Exception as e:
            pass
          finally:
            pass
    result.sort(key=lambda tup: tup[2], reverse=True)
    print('Para a palavra ' + word)
    print("1° {palavra} | texto: {text} | similaridade de {sim:.2f}".format(palavra = result[0][0], text = result[0][1], sim = result[0][2]) )
    id = 1
    while(result[0][0] == result[id][0]):
      id +=1
    print("2° {palavra} | texto: {text} | similaridade de {sim:.2f}".format(palavra = result[id][0], text = result[id][1], sim = result[id][2]) )
    id2 = id + 1
    while(result[id2][0] == result[id][0]):
      id2 +=1
    print("3° {palavra} | texto: {text} | similaridade de {sim:.2f}".format(palavra = result[id2][0], text = result[id2][1], sim = result[id2][2]) )
    print("----------------------------------------------------------------")

    ## Médio
  for word in words_to_compare:
    result = []
    for docum in range(0, len(data)): ##Por documento
      for vects in range(0, len(data[docum][1])): ## Vetor de vetores de palavras
        diff = 0
        for arr in range(0, len(data[docum][1][vects])): ## Vetor de palavra
          try:
            word_acc = data[docum][1][vects][arr] ## palavra
            diff += model[0].similarity(word, word_acc)
          except Exception as e:
            pass
          finally:
            pass
        if(len(data[docum][1][vects]) > 0):
          result.append((data[docum][1][vects], docum, (diff/len(data[docum][1][vects]))))
    result.sort(key=lambda tup: tup[2], reverse=True)
    print('Para a palavra ' + word + ' Vector Medios')
    print("1° {palavra} | texto: {text} | similaridade de {sim:.2f}".format(palavra = result[0][0], text = result[0][1], sim = result[0][2]) )
    print("2° {palavra} | texto: {text} | similaridade de {sim:.2f}".format(palavra = result[1][0], text = result[1][1], sim = result[1][2]) )
    print("3° {palavra} | texto: {text} | similaridade de {sim:.2f}".format(palavra = result[2][0], text = result[2][1], sim = result[2][2]) )
    print("----------------------------------------------------------------")





***************** Resultados Com 50 *****************


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Para a palavra walmart
1° retailer | texto: 295 | similaridade de 0.72
2° stores | texto: 18 | similaridade de 0.69
3° tesco | texto: 50 | similaridade de 0.68
----------------------------------------------------------------
Para a palavra pineapple
1° juice | texto: 42 | similaridade de 0.81
2° plum | texto: 232 | similaridade de 0.78
3° honey | texto: 30 | similaridade de 0.72
----------------------------------------------------------------
Para a palavra platypus
1° hamster | texto: 46 | similaridade de 0.62
2° feline | texto: 125 | similaridade de 0.57
3° qrio | texto: 25 | similaridade de 0.57
----------------------------------------------------------------
Para a palavra python
1° mouse | texto: 0 | similaridade de 0.63
2° scripting | texto: 149 | similaridade de 0.60
3° ape | texto: 174 | similaridade de 0.59
----------------------------------------------------------------
Para a palavra javascript
1° scripting | texto: 149 | similaridade de 0.80
2° xml | texto: 153 | similarida

# Segunda questão

In [ ]:
# Problema: quais textos são sobre segurança tecnológica? (yes - no)
# Rotulando os dados
import os
import glob
import nltk
import random
import pandas as pd
import random
import re
import string
from sklearn.feature_extraction.text import CountVectorizer

path1_yes = '/content/yes_sec' # 47 arquivos
path2_no = '/content/no_sec' # 53 arquivos

txt_corpus = []

# Lendo pasta 1
for filename in glob.glob(os.path.join(path1_yes, '*.txt')):
	with open(filename, 'r') as f_in:
		txt_corpus.append([filename.split('/')[-1].split('.')[0], f_in.read()])

df_corpus = pd.DataFrame.from_records(txt_corpus)
df_corpus.columns = ["id", "content"]

# Lendo pasta 2
for filename in glob.glob(os.path.join(path2_no, '*.txt')):
	with open(filename, 'r') as f_in:
		txt_corpus.append([filename.split('/')[-1].split('.')[0], f_in.read()])

df_corpus = pd.DataFrame.from_records(txt_corpus)
df_corpus.columns = ["id", "content"]

In [ ]:
#from sklearn.utils import shuffle
#df_merge = shuffle(df_merge)
#df_merge.head(5)
df_corpus.insert(loc=0,column="label",value="")
for i in range(0,47):
  df_corpus['label'][i] = "1" # Yes

for i in range(47,100):
  df_corpus['label'][i] = "0" # No

In [ ]:
# STEP 1 - PREPARANDO OS DADOS
# Removendo as palavras que contém numeros
df_corpus['content'] = df_corpus['content'].replace(to_replace=r'\w*\d\w*', value='', regex=True)
df_corpus.head(5)

,label,id,content
0,1,055,Junk e-mails on relentless rise\n\nSpam traffi...
1,1,022,Sun offers processing by the hour\n\nSun Micro...
2,1,188,Call for action on internet scam\n\nPhone comp...
3,1,031,Solutions to net security fears\n\nFake bank e...
4,1,136,BT offers free net phone calls\n\nBT is offeri...


In [ ]:
# Removendo pontuação e colocando tudo em minúsculo
def remove_punctuation(text):
  return  re.sub('[%s]' % re.escape(string.punctuation), ' ', text)

df_corpus['content'] = df_corpus['content'].apply(remove_punctuation) # Removendo pontuação

df_corpus.content = df_corpus.content.str.lower()
df_corpus.content

0     junk e mails on relentless rise\n\nspam traffi...
1     sun offers processing by the hour\n\nsun micro...
2     call for action on internet scam\n\nphone comp...
3     solutions to net security fears\n\nfake bank e...
4     bt offers free net phone calls\n\nbt is offeri...
                            ...                        
95    commodore finds new lease of life\n\nthe once ...
96    progress on new internet domains\n\nby early  ...
97    when invention turns to innovation\n\nit is un...
98    pompeii gets digital make over\n\nthe old fash...
99    broadband in the uk growing fast\n\nhigh speed...
Name: content, Length: 100, dtype: object

In [ ]:
# Embaralhando os dados
from sklearn.utils import shuffle

df_corpus = shuffle(df_corpus)
df_corpus.head(5)

,label,id,content
85,0,014,eu software patent law faces axe\n\nthe europe...
34,1,164,who do you think you are \n\nthe real danger i...
88,0,112,portable playstation ready to go\n\nsony s pla...
67,0,001,ink helps drive democracy in asia\n\nthe kyrgy...
68,0,025,sony psp console hits us in march\n\nus gamers...


In [ ]:
df_corpus_noid = df_corpus
del df_corpus_noid['id']
print(df_corpus_noid.head(5))

   label                                            content
85     0  eu software patent law faces axe\n\nthe europe...
34     1  who do you think you are \n\nthe real danger i...
88     0  portable playstation ready to go\n\nsony s pla...
67     0  ink helps drive democracy in asia\n\nthe kyrgy...
68     0  sony psp console hits us in march\n\nus gamers...


In [ ]:
# STEP 2 - SPLIT NOS DADOS
from sklearn.model_selection import train_test_split

#x = df_corpus.content
#y = df_corpus.label

#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)
train, test = train_test_split(df_corpus_noid, test_size=0.33, random_state=42)

print("Train: ",train.head())
print("Test: ", test.head())

Train:     label                                            content
47     0  china net cafe culture crackdown\n\nchinese au...
40     1   evil twin  fear for wireless net\n\npeople us...
55     0  gadget show heralds  season\n\npartners of tho...
97     0  when invention turns to innovation\n\nit is un...
10     1  screensaver tackles spam websites\n\nnet users...
Test:     label                                            content
83     0  global blogger action day called\n\nthe global...
28     1  concern over rfid tags\n\nconsumers are very c...
43     1  technology gets the creative bug\n\nthe hi tec...
69     0  blogger grounded by her airline\n\na us airlin...
64     0  reboot ordered for eu patent law\n\na european...


In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import utils
import csv
from tqdm import tqdm
import multiprocessing
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#df12 = df['content'].apply(tokenize_text)
#df12.tolist()
train_tagged = train.apply(lambda r: TaggedDocument(words=tokenize_text(r['content']), tags=[r.label]), axis=1)
test_tagged = test.apply(lambda r: TaggedDocument(words=tokenize_text(r['content']), tags=[r.label]), axis=1)

train_tagged.values[30]

TaggedDocument(words=['broadband', 'in', 'the', 'uk', 'growing', 'fast', 'high', 'speed', 'net', 'connections', 'in', 'the', 'uk', 'are', 'proving', 'more', 'popular', 'than', 'ever', 'bt', 'reports', 'that', 'more', 'people', 'signed', 'up', 'for', 'broadband', 'in', 'the', 'last', 'three', 'months', 'than', 'in', 'any', 'other', 'quarter', 'the', 'connections', 'take', 'the', 'total', 'number', 'of', 'people', 'in', 'the', 'uk', 'signing', 'up', 'for', 'broadband', 'from', 'bt', 'to', 'almost', 'million', 'nationally', 'more', 'than', 'million', 'browse', 'the', 'net', 'via', 'broadband', 'britain', 'now', 'has', 'among', 'the', 'highest', 'number', 'of', 'broadband', 'connections', 'throughout', 'the', 'whole', 'of', 'europe', 'according', 'to', 'figures', 'gathered', 'by', 'industry', 'watchdog', 'ofcom', 'the', 'growth', 'means', 'that', 'the', 'uk', 'has', 'now', 'surpassed', 'germany', 'in', 'terms', 'of', 'broadband', 'users', 'per', 'people', 'the', 'uk', 'total', 'of', 'milli

In [ ]:
#tagged_data = [TaggedDocument(words=df12, tags=[str(i)]) for i, _d in enumerate(df12)]
#print(tagged_data[1])
# Construindo o vocabulário - Há o DBOW e o DM:
# Usando o DBOW
import multiprocessing

my_cores = multiprocessing.cpu_count()

model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=my_cores)
model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])

100%|██████████| 67/67 [00:00<00:00, 183912.54it/s]


In [ ]:
#cores = multiprocessing.cpu_count()
#model_dbow = Doc2Vec(tagged_data, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, workers=cores, alpha=0.025, min_alpha=0.001)
#model_dbow.build_vocab([x for x in tqdm(tagged_data)])
# Treinando o Doc2Vec - DBOW
import time

# Tempo de execução
def seconds_transform(seconds_time):
  hours = int(seconds_time/3600)
  rest_1 = seconds_time%3600
  minutes = int(rest_1/60)
  seconds = rest_1 - 60*minutes
  #print(seconds)
  print("Time: ", (hours), "h ", (minutes), "min ", round(seconds,2), " s")

srt_time_doc2vec = time.time() # Time start
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), 
                     total_examples=len(train_tagged.values), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

seconds_doc2vec = (time.time() - srt_time_doc2vec) # Time end
seconds_transform(seconds_doc2vec)

100%|██████████| 67/67 [00:00<00:00, 19103.90it/s]


Time:  0 h  0 min  1.9  s


In [ ]:
# Construindo a função para versão final de vetorização da feature
def vec_for_learning(model, tagged_docs):
  sents = tagged_docs.values
  targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
  return targets, regressors

y_train, x_train = vec_for_learning(model_dbow, train_tagged)
y_test, x_test = vec_for_learning(model_dbow, test_tagged)

In [ ]:
#print([x for x in tqdm(tagged_data)])
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB # Naive Bayes
from sklearn.tree import DecisionTreeClassifier # DTC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

#n_cv = int(7) # LR
lr = LogisticRegression()
lr.fit(x_train, y_train)
y_pred_lr = lr.predict(x_test)
# Naive Bayes
nb = GaussianNB()
nb.fit(x_train, y_train)
y_pred_nb = nb.predict(x_test)
# Decision Tree Classifier
dt = DecisionTreeClassifier()
dt.fit(x_train, y_train)
y_pred_dt = dt.predict(x_test)
#accur_lr = cross_val_score(lr, x_train, y_train, cv=n_cv, scoring='accuracy')
#precision_lr = cross_val_score(lr, x_train, y_train, cv=n_cv, scoring='precision')
#recall_lr = cross_val_score(lr, x_train, y_train, cv=n_cv, scoring='recall')
#precision_lr = cross_val_score(lr, x_train, y_train, cv=n_cv, scoring='f1')
#print("Doc2Vec - Acurácia LR: ", round(accur_lr.mean(),2)) # Acurácia esperada no modelo
#print("Doc2Vec - Precision LR: ", round(precision_lr.mean(),2)) # Precision esperada no modelo
#print("Doc2Vec - Recall LR: ", round(precision_lr.mean(),2)) # Recall esperada no modelo
#print("Doc2Vec - F1 Score LR: ", round(precision_lr.mean(),2)) # F1 Score esperada no modelo


In [ ]:
# LR
precision_lr = precision_score(y_test, y_pred_lr, pos_label='1', average='binary')
recall_lr = recall_score(y_test, y_pred_lr, pos_label='1', average='binary')
f1_lr = f1_score(y_test, y_pred_lr, pos_label='1', average='binary')

# NB
precision_nb = precision_score(y_test, y_pred_nb, pos_label='1', average='binary')
recall_nb = recall_score(y_test, y_pred_nb, pos_label='1', average='binary')
f1_nb = f1_score(y_test, y_pred_nb, pos_label='1', average='binary')

print("Accuracy LR:....... ", round(lr.score(x_test, y_test).mean(),2)) # Pode ser também accuracy_score(y_test, y_pred)
print("Accuracy NB:....... ", round(nb.score(x_test, y_test).mean(),2)) # Pode ser também accuracy_score(y_test, y_pred)
print("Accuracy DT:....... ", round(dt.score(x_test, y_test).mean(),2)) # Pode ser também accuracy_score(y_test, y_pred)
print("Accuracy CV LR:.... 0.67")
print("Accuracy TF-IDF LR: 0.73")
print("Accuracy CV NB:.... 0.7")
print("Accuracy TF-IDF NB: 0.73")
print("Accuracy CV DT:.... 0.67")
print("Accuracy TF-IDF DT: 0.67")
print("")
print("Precision LR:....... ", round(precision_lr.mean(),2))
print("Precision NB:....... ", round(precision_lr.mean(),2))
print("Precision DT:....... ", round(precision_lr.mean(),2))
print("Precision CV LR:.... 0.5")
print("Precision TF-IDF LR: 0.56")
print("Precision CV NB:.... 0.53")
print("Precision TF-IDF NB: 0.56")
print("Precision CV DT:.... 0.5")
print("Precision TF-IDF DT: 0.5")
print("")
print("Recall LR:....... ", round(recall_lr.mean(),2))
print("Recall NB:....... ", round(recall_lr.mean(),2))
print("Recall DT:....... ", round(recall_lr.mean(),2))
print("Recall CV LR:.... 0.5")
print("Recall TF-IDF LR: 0.9")
print("Recall CV NB:.... 0.8")
print("Recall TF-IDF NB: 1.0")
print("Recall CV DT:.... 0.7")
print("Recall TF-IDF DT: 0.6")
print("")
print("F1 Score LR:....... ", round(f1_lr.mean(),2))
print("F1 Score NB:....... ", round(f1_lr.mean(),2))
print("F1 Score DT:....... ", round(f1_lr.mean(),2))
print("F1 Score CV LR:.... 0.5")
print("F1 Score TF-IDF LR: 0.69")
print("F1 Score CV NB:.... 0.5")
print("F1 Score TF-IDF NB: 0.64")
print("F1 Score CV DT:.... 0.58")
print("F1 Score TF-IDF DT: 0.55")
#print("Testing AS: ", round(accuracy_score(y_test, y_pred),2))

Accuracy LR:.......  0.48
Accuracy NB:.......  0.36
Accuracy DT:.......  0.58
Accuracy CV LR:.... 0.67
Accuracy TF-IDF LR: 0.73
Accuracy CV NB:.... 0.7
Accuracy TF-IDF NB: 0.73
Accuracy CV DT:.... 0.67
Accuracy TF-IDF DT: 0.67

Precision LR:.......  0.42
Precision NB:.......  0.42
Precision DT:.......  0.42
Precision CV LR:.... 0.5
Precision TF-IDF LR: 0.56
Precision CV NB:.... 0.53
Precision TF-IDF NB: 0.56
Precision CV DT:.... 0.5
Precision TF-IDF DT: 0.5

Recall LR:.......  0.33
Recall NB:.......  0.33
Recall DT:.......  0.33
Recall CV LR:.... 0.5
Recall TF-IDF LR: 0.9
Recall CV NB:.... 0.8
Recall TF-IDF NB: 1.0
Recall CV DT:.... 0.7
Recall TF-IDF DT: 0.6

F1 Score LR:.......  0.37
F1 Score NB:.......  0.37
F1 Score DT:.......  0.37
F1 Score CV LR:.... 0.5
F1 Score TF-IDF LR: 0.69
F1 Score CV NB:.... 0.5
F1 Score TF-IDF NB: 0.64
F1 Score CV DT:.... 0.58
F1 Score TF-IDF DT: 0.55


**2 - b)**

In [ ]:
# 4a lista - 2a questão - letra a)